<h1 style="color:blue">Praktikum 9</h1>
<h3 style="color:blue">Osalausestamine ja verbiahelate analüüs.<br> Andmete serialiseerimine</h3>

Käesolev praktikum on pühendatud kahele EstNLTK keeleanalüüsi komponendile: osalausestajale ja verbiahelate tuvastajale.
Osalausestamine on täienduseks süntaksianalüüsile ja võimaldab nt tuvastada lihtsamaid komavigu; verbiahelate analüüs aga võimaldab uurida tekstis olevaid väiteid (nt eristada jaatavaid väiteid eitavatest ja kindlaid väiteid võimalikest).

Kõige lõpus on lisamaterjal, mis käsitleb Pythoni mälus olevate andmete otsesalvestamist ehk serialiseerimist.

NB! Selles praktikumis läheb tarvis ka teadmisi XML sisendi lugemise kohta, seega tasub kõrvale lahti võtta 7. praktikumi asjakohased materjalid.

## Osalausete tuvastamine

Osalausestaja on programm, mille abil saab pikad ja keeruka struktuuriga loomuliku keele laused jagada väiksemateks/lihtsamateks juppideks: osalauseteks. Enamasti ongi osalaused lihtlause-sarnased: sisaldavad ühte peaverbi (finiitverbi) ning sellega süntaktiliselt seotud sõnu. 

Sissejuhatuseks vaatame kõigepealt üht näidet osalausestaja kasutamise kohta:

In [1]:
from estnltk import Text

text = 'Igaüks, kes traktori eest miljon krooni lauale laob, on huvitatud sellest, et traktor '+\
       'meenutaks lisavõimaluste poolest võimalikult palju kosmoselaeva.'

# Osalausestame teksti:
text_w_clauses = Text(text).tag_clauses()

# Kõik osalaused järjendina
text_w_clauses.clause_texts

['Igaüks on huvitatud sellest,',
 ', kes traktori eest miljon krooni lauale laob,',
 'et traktor meenutaks lisavõimaluste poolest võimalikult palju kosmoselaeva.']

 * _Tehniline märkus:_ nagu ajaväljendite tuvastaja ja süntaksianalüsaator, nii kasutab ka EstNLTK osalausestaja `java`-t. Seetõttu läheb esmakordsel `tag_clauses()` väljakutsel tavaliselt natukene rohkem aega, kuna toimub `java` komponendi initsialiseerimine;

Automaatne osalausete tuvastaja eristab kahte liiki osalauseid:
   - tavalised osalaused, mis on tüüpiliselt üksteisest eraldatud koma ja/või sidesõnade (_ja, ning, et, sest, kuid_ vms) abil; eelmises näites on tavalised osalaused _'Igaüks on huvitatud sellest'_ ja _'et traktor meenutaks lisavõimaluste poolest võimalikult palju kosmoselaeva.'_;
   - kiillaused, mis on sisestatud mingi lause või osalause sisse ning jagavad selle kaheks tükiks\*; eelmises näites: _'kes traktori eest miljon krooni lauale laob'_ on kiillause lause _'Igaüks on huvitatud sellest'_ sees; 
     
     \* NB! Osalausestaja kiillausete käsitlus ei ühti eesti keele traditsiooniliste grammatikakirjelduste EKG II ja [EKK](http://www.eki.ee/books/ekk09/index.php?id=482&p=5&p1=3) kiillausete definitsiooniga. Milliseid süntaktilisi üksusi ja miks osalausestajas kiillauseteks nimetatakse, seda selgitab detailsemalt programmi kirjeldav [Kaalepi ja Muischneki artikkel]( http://arhiiv.rakenduslingvistika.ee/ajakirjad/index.php/aastaraamat/article/view/ERYa8.04/6)

Milleks on osalausete tuvastajat tarvis? Osalausestatud tekstide peal on võimalik uurida mitmeid keelenähtuseid, mida automaatne  süntaksianalüüs veel lahendada ei oska või mille lahendus vajab täiustamist. 

Üheks selliseks keelenähtuseks on _kollokatsioonid_ ehk sõnade koosesinemised, millega sageli kaasneb ka eraldiseisev/metafoorne tähendus (nt _'hea seisma'_, _'hambasse puhuma'_, _'õppust võtma'_). Kollokatsioonide automaatne tuvastamine nõuab kõigepealt, et piiritleme konteksti, mille seest kollokatsioonikandidaate otsime. Kui kontekstiks valida terve lause, siis liitlausete puhul jääb sõelale üksjagu "prahti" ehk siis selliseid sõnapaare, mis ületavad osalausepiire ning ei saa olla kollokatsioonid. Seega saab osalausestamise tulemust aluseks võttes leida täpsemad kollokatsioonikandidaadid.

Teiseks keelenähtuseks on sõnadevahelised _rektsiooniseosed_ ehk süntaksiseosed, kus üks sõna määrab teise olemasolu ja vormi lauses. Näiteks, verb _'hoolima'_ tingib sageli seestütlevas käändes sõna esinemise lauses (_'Kas sa hoolid **temast** ?'_) ning verb _'tutvustama'_ osastavas ja alaleütlevas käändes sõnad (_'Konsul tutvustab **kuurorti** ka oma **kaasmaalastele**.'_). Ka rektsiooniseoste uurimisel aitab osalausestamine täpsemalt piiritleda konteksti, mille sees seosed võivad sõnade vahel esineda (rektsiooniseosed tüüpiliselt ei ületa osalausepiire), ning seega parandada analüüsi kvaliteeti. Verbiahelate tuvastamine, mida uurime praktikumi teises pooles, toetub sisuliselt samuti osalausepiiride märgendusele ja otsib verbiahelaid osalausete seest.

Jätkame nüüd tehnilisemal lainel. 

Kuidas leida, millistesse osalausetesse sõnad kuuluvad? Pärast automaatset  osalausestamist lisatakse iga sõna külge (ehk siis: `'words'` kihti `Text` objektil) atribuut nimega `'clause_index'`, milles tuuakse välja sõnaga seotud osalause identifikaator:

In [2]:
from estnltk import Text

text = 'Igaüks, kes traktori eest miljon krooni lauale laob, on huvitatud sellest, et traktor meenutaks '+\
       'lisavõimaluste poolest võimalikult palju kosmoselaeva.'
text_w_clauses = Text(text).tag_clauses()

In [3]:
clause_indices = [word['clause_index'] for word in text_w_clauses['words']]
clause_indices

[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2]

* Oluline on meeles pidada, et osalausete identifikaatorid on unikaalsed vaid ühe lause piires. Ehk siis: kui meil on tekstis rohkem kui üks lause, siis iga lause sees alustatakse osalausete loendamist jälle 0-ist;

Kuidas uurida osalausete struktuuri (tavalised osalaused vs kiillaused)? Lisaks atribuudile `'clause_index'` lisatakse sõnade külge atribuut nimega `'clause_annotation'`, kus tuuakse välja sõnaga seotud osalausepiiri liik (kui sõnaga on seotud osalausepiir):

In [4]:
clause_boundaries = [word['clause_annotation'] if 'clause_annotation' in word else '' for word in text_w_clauses['words']]
clause_boundaries

['',
 'embedded_clause_start',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'embedded_clause_end',
 '',
 '',
 '',
 'clause_boundary',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

Seega on võimalik `zip()` funktsiooni abil siduda sõnad vastavate osalauseindeksite/osalausepiiri tähistega:

In [5]:
words = [word['text'] for word in text_w_clauses['words']]

# Tekstisõned ja nendega seotud osalauseidentifikaatorid
list( zip(clause_indices, words) ) 

[(0, 'Igaüks'),
 (1, ','),
 (1, 'kes'),
 (1, 'traktori'),
 (1, 'eest'),
 (1, 'miljon'),
 (1, 'krooni'),
 (1, 'lauale'),
 (1, 'laob'),
 (1, ','),
 (0, 'on'),
 (0, 'huvitatud'),
 (0, 'sellest'),
 (0, ','),
 (2, 'et'),
 (2, 'traktor'),
 (2, 'meenutaks'),
 (2, 'lisavõimaluste'),
 (2, 'poolest'),
 (2, 'võimalikult'),
 (2, 'palju'),
 (2, 'kosmoselaeva'),
 (2, '.')]

In [6]:
# Tekstisõned ja nendega seotud osalausepiiri märgendid
list( zip(words, clause_boundaries) ) 

[('Igaüks', ''),
 (',', 'embedded_clause_start'),
 ('kes', ''),
 ('traktori', ''),
 ('eest', ''),
 ('miljon', ''),
 ('krooni', ''),
 ('lauale', ''),
 ('laob', ''),
 (',', 'embedded_clause_end'),
 ('on', ''),
 ('huvitatud', ''),
 ('sellest', ''),
 (',', 'clause_boundary'),
 ('et', ''),
 ('traktor', ''),
 ('meenutaks', ''),
 ('lisavõimaluste', ''),
 ('poolest', ''),
 ('võimalikult', ''),
 ('palju', ''),
 ('kosmoselaeva', ''),
 ('.', '')]

Vajadusel vt ka osalausestajat tutvustavat abimaterjali [EstNLTK dokumentatsioonis](https://estnltk.github.io/estnltk/1.4.1/tutorials/text.html#tagging-clauses).

### Osalausestamine ja komavead

Automaatne osalausestaja eeldab vaikimisi, et ortograafiareegleid on järgitud täpselt ning komad paiknevad seal, kus nad peaksid olema. Seega, kui analüüsitavast tekstist on mõni osalauseid eraldav koma puudu, siis kannatab ka osalausestamise kvaliteet.

Siiski sisaldab osalausestaja ka eksperimentaalset režiimi, mille puhul üritatakse osalausepiire määrata ka siis, kui komad on puudu. Et seda režiimi kasutada, tuleb `Text` objekti vaikeosalausestaja kirjutada üle uue osalausestajaga (`ClauseSegmenter`), millel on sisse lülitatud komavigade-suhtes-vähetundlikkuse lipp (`ignore_missing_commas=True`). Täpsemalt käib see nii:

In [7]:
from estnltk import Text
from estnltk import ClauseSegmenter

# Loome teksti ja anname sellele kaasa komavigade-suhtes-vähetundliku osalausestaja
segmenter = ClauseSegmenter( ignore_missing_commas=True )
text = Text('Keegi teine ka siin ju kirjutas et ütles et saab ise asjadele järgi minna aga vastust seepeale ei tulnudki.', \
            clause_segmenter = segmenter)

# Osalausestame
text = text.tag_clauses()

# Kuvame teksti osalause kaupa
for clause in text.split_by('clauses'):
    print (clause.text)

Keegi teine ka siin ju kirjutas
et ütles
et saab ise asjadele järgi minna
aga vastust seepeale ei tulnudki.


### Ülesanne 1. Võimalike komavigade tuvastamine (2 p)

Proovime nüüd kasutada osalausestajat komavigade tuvastamiseks. 

Kataloogis `'0_foorumid_xml'` on TÜ koondkorpuse [uue meedia alamkorpusest](http://www.cl.ut.ee/korpused/segakorpus/uusmeedia) pärinevad Planet foorumite postituste salvestused. 
Igas XML failis on ühe alamfoorumi teemad; teemad on jagatud postitusteks ning postitused omakorda lauseteks.

Looge osalausestaja abil programm, mis tuvastab foorumi teemadest (kataloogis `'0_foorumid_xml'`) võimalikke komavigu. Programm võrdleb osalausestamise tavarežiimi tulemust komavigade-suhtes-vähetundliku režiimi tulemusega ning leiab, kas tulemused erinevad. 

Programm raporteerib, millise teema millisel leheküljel on kõige rohkem "potentsiaalseid komavigasid" (st osalausepiiride märgendid erinevad kahte režiimi kasutades kõige rohkem), ning ühtlasi väljastab potentsiaalsete komavigade kohad (tollel leheküljel). 
Väljund peaks olema selline, et programmi kasutajal on konteksti põhjal võimalik otsustada, kas tegu on komaveaga või "valealarmiga". 
Selleks tuleb väljastada / märgistada sõnad, millel osalausemärgendus erines, ning lisada kontekst ulatuses *N* sõna enne ja pärast, sh ka üle lausepiiride ulatuv kontekst. 
Konteksti suurus _N_ valige ise, aga see võiks olla muutujas, nii et seda saaks lihtsasti muuta.

Detaile:

* Kasutage tekstide sisselaadimiseks on EstNLTK funktsiooni `parse_tei_corpora` või `parse_tei_corpus` (tutvustatud 7. praktikumis). Tekste tuleks sisse lugeda teemade kaupa ehk siis määrata funktsiooni parameetriks `xml_target=['teema']`. Kindlasti tuleks määrata ka kodeering ('utf-8').  Sisseloetud tekstid on varustatud ka meta-andmetega, uurige `Text` objektide võtmeid. Nii teema nime kui lehekülje numbri saab kätte sama võtme alt;
* Erinevuste leidmiseks tuleb igast tekstist kõigepealt luua kaks osalausestatud versiooni: tavalise osalausestajaga töödeldud versioon ning "komavigade-suhtes-vähetundliku" osalausestajaga töödeldud versioon; Seejärel tuleks versioone võrrelda sõna-sõna haaval ning leida osalausepiiride erinevused;
* Konteksti väljastamine: ilmselt tuleb igal sammul teada sõna positsiooni/indeksit tekstis; kui olete tuvastanud potentsiaalse komakoha, tehke kindlaks, kui suures ulatuses sõna konteksti saab kuvada (nt teksti 2. positsioonil ei saa kuvada 10 eelnevat sõna, saab ainult 2), noppige konteksti jäävad sõnad tekstist välja ja printige ekraanile;

## Verbiahelate tuvastamine

Kui lause eesmärgiks on mingi väite väljendamine, siis selle väite _faktilisust_ / _modaalsust_ mõjutab sageli see, milline on väidet väljendav _verbiahel_. 
Näiteks, verbiahelaid uurides võib eristada jaatavaid väiteid eitavatest (nt _'Peaminister **astub** homme tagasi'_ vs _'Peaminister eile siiski tagasi **ei astunud**'_), kindlaid väiteid võimalikest (nt _'Ta **tuleb** homme'_ vs _'Ta **võib tulla** homme'_) ning kindlaid tegevusi soovidest / plaanidest / üritamistest jms (nt 'Juku **alustas** kodutööga' vs 'Juku **kavatseb** kodutööga **alustada**', 'Juku **tahab** kodutööd **alustada**', 'Juku kinnitusel **on võimalik** kodutööd **alustada**'). 
Verbiahelate tuvastaja leiabki verbidega seotud ühe- ja mitmesõnalisi üksuseid, millele toetudes saab üles ehitada lause _faktilisuse_ / _modaalsuse_ automaatse analüüsi.

Lihtsaimaks kasutusnäiteks on jaatavate ja eitavate verbiahelate tuvastamine:

In [8]:
from estnltk import Text

text = Text('Kas Juku alustas kodutööga? Minuteada ei alustanud.')

# Tuvastame verbiahelad 
text = text.tag_verb_chains()

# Väljastame verbiahelad (vastavad tekstid)
text.verb_chain_texts

['alustas', 'ei alustanud']

_Ühe- vs mitmesõnalised ahelad._ Nagu eelmisest näitest võib näha, siis jaatav "verbiahel" võib koosneda ainult ühest sõnast. Seepärast võib nimi "verbiahel" esmapilgul tunduda veidi eksitav (mismoodi on üks verb ahel?!), aga kuna verbiahelate tuvastaja eesmärgiks on süstemaatiliselt leida üles kõik lause / osalause peaverbe¹ hõlmavad konstruktsioonid ja peaverbid ongi sageli ühesõnalised, siis tagastab ka tuvastaja nii mitmesõnalisi kui ka ühesõnalisi tekstiüksuseid;

 * ¹ _peaverbi_ all mõeldakse siin enamasti finiitverbi ehk pöördelist verbivormi. Aga on ka erandeid, nende kohta vt järgnevat loendit;

Konkreetsemalt katab verbiahelate tuvastaja järgmisi konstruktsioone:
   * Lause/osalause peaverbid:
       * Jaatavad verbid, v.a. _olema_ (nt 'Pidevalt **uurivad** asjade seisu ka hollandlased');
       * Ühe- ja mitmesõnalised jaatavad _olema_-konstruktsioonid (nt 'Raha **on** alati vähe', '**Oleme** sellist kino ennegi **näinud**');
       * Eitusega peaverbid: _ei/ära/pole/ega + verb_ (nt 'Helistasin korraks Carmenile, kuid ta **ei vastanud**');    
   * Peaverbide laiendused:
       * _verb + infiniitverb_ ahel, kus _infiniitverbi_ ja eelneva verbi vahel on rektsiooniseos, nt verb _kutsuma_ saab võtta _ma_-verbidest argumente ('Kevadpäike **kutsub** mind **suusatama**') ning _püüdma_ saab võtta _da_-verbidest argumente ('Aita **ei püüdnudki** Leenat **mõista**');
       * _verb + nom/adv + infiniitverb_ ahel, kus mitmesõnaline üksus _verb + nimisõna/adverb_ on rektsiooniseoses sellele järgneva infiniitverbiga; näiteks, verb _otsima_ saab moodustada mitmesõnalise üksuse sõnaga _võimalust_ ning see üksus saab võtta omakorda _da_-verbe argumentideks ('Seepärast **otsisimegi** **võimalust** kusagilt mõned ilvesed **hankida**');

Kui rektsiooniseoste tuvastamisel ei teki mitmesusi (st ei tuvastata mitut erinevat viisi ühe ahela laiendamiseks), võib verbiahelate tuvastaja algoritm laiendada üht ahelat ka mitu korda ning moodustada üksjagu pika ahela, nt lausest _'Minul oleks pidanud olema õigus ise endale külalisi kutsuda'_ leitakse ahel  **oleks** => **pidanud** => **olema** => **õigus** => **kutsuda**.

In [9]:
from estnltk import Text

text = Text('Juku tahtis kodutööd juba ammu alustada, aga ei saanud. Juku kinnitusel on siiski võimalik kodutööd alustada.')

# Märgendame verbiahelad 
text = text.tag_verb_chains()

# Väljastame verbiahelad (vastavad tekstid)
text.verb_chain_texts

['tahtis alustada', 'ei saanud', 'on võimalik alustada']

Tekstist leitud verbiahelate detailse info saab kätte atribuudist `verb_chains`:

In [10]:
# Kogu informatsioon verbiahelate kohta:
text.verb_chains

[{'analysis_ids': [[0], [0]],
  'clause_index': 0,
  'end': [11, 39],
  'mood': 'indic',
  'morph': ['V_s', 'V_da'],
  'other_verbs': False,
  'pattern': ['verb', 'verb'],
  'phrase': [1, 5],
  'pol': 'POS',
  'roots': ['taht', 'alusta'],
  'start': [5, 31],
  'tense': 'imperfect',
  'voice': 'personal'},
 {'analysis_ids': [[0], [0]],
  'clause_index': 1,
  'end': [47, 54],
  'mood': 'indic',
  'morph': ['V_neg', 'V_nud'],
  'other_verbs': False,
  'pattern': ['ei', 'verb'],
  'phrase': [8, 9],
  'pol': 'NEG',
  'roots': ['ei', 'saa'],
  'start': [45, 48],
  'tense': 'imperfect',
  'voice': 'personal'},
 {'analysis_ids': [[0, 1], [0], [0]],
  'clause_index': 0,
  'end': [74, 90, 108],
  'mood': 'indic',
  'morph': ['V_b/vad', 'A_sg n', 'V_da'],
  'other_verbs': False,
  'pattern': ['ole', 'nom/adv', 'verb'],
  'phrase': [2, 4, 6],
  'pol': 'POS',
  'roots': ['ole', 'võimalik', 'alusta'],
  'start': [72, 82, 100],
  'tense': 'present',
  'voice': 'personal'}]

Nagu teiste fraasimärgenduste puhul, nii on ka verbiahelate märgendused esitatud sõnastike kujul. Käesoleva praktikumi seisukohast on kõige olulisem info järgmiste võtmete all:

* `'pol'` -- kas verbiahel on jaatav (`'POS'`) või eitav (`'NEG'`)? Võimalik ka väärtus `'??'` (määramata);
* `'roots'` -- järjend ahela sõnade algvormidega (morf analüüsi `'root'` atribuudid);
* `'mood'` -- ahela finiitverbi kõneviis. Võimalikud väärtused: `'indic'` (kindel kv), `'imper'` (käskiv ja möönev), `'condit'` (tingiv), `'quotat'` (kaudne),  või `'??'` (määramata);
* `'tense'` -- ahela finiitverbi/konstruktsiooni grammatiline aeg. Võimalikud väärtused sõltuvad kõneviisist. Kindlas kõneviisis: `'present'`, `'imperfect'`, `'perfect'`, `'pluperfect'`, käskivas kõneviisis:  `'present'` ning tingivas ja kaudses: `'present'` ja `'past'`. Lisaks võib jääda ka määramata (`'??'`);
* `'voice'` -- ahela finiitverbi tegumood. Võimalikud väärtused: `'personal'`, `'impersonal'`, `'??'`;

Erinevalt teistest fraasimärgendustest (nt ajaväljenditest, nimeüksustest) võivad verbiahelate fraasid olla katkendlikud / sisaldada lünkasid. Seetõttu pole ka verbiahelate asukohti kirjeldavad atribuudid `'start'` ja `'end'` mitte tavalised arvud, vaid arvude järjendid, milles iga järjendi liige kirjeldab ühe ahela liikme vastavalt kas siis algust või lõpp-positsiooni tekstis.

**Sõnade järjekord ahelas. Tähenduse mõjutajad ja sisuverbid.** Võtme `'roots'` all olevad algvormid ei ole organiseeritud mitte nende lauses esinemise järjekorras, vaid järjekorras, mis peegeldab sõnadevahelisi seoseid: 
iga järgnev sõna ahelas on eelmise sõna süntaktiline alluv.
Sellise järjestamise tulemusena on ahela eesotsas _tähendust mõjutavad verbid_, nt eituse abiverb või modaalverb, ning ahela lõpus _sisuverb_ ehk siis verb, mis annab edasi väite põhisisu.
Näide:

In [11]:
text = Text('Ja teda kutsuda ei proovitudki?')
# Väljastame verbiahelate algvormid
text.verb_chains[0]['roots']

['ei', 'proovi', 'kutsu']

Sama järjestust kasutatakse ka järjendites `'morph'` ja `'pattern'`. Erandlikult on järjendites `'start'` ja `'end'` asukohad kirjeldatud lauses esinemise järjekorras ning ka mugavusmeetod `verb_chain_texts` paneb sõnad kokku lauses esinemise järjekorras.

Veel infot verbiahelate tuvastaja kohta leiab [EstNLTK abimaterjalist](https://estnltk.github.io/estnltk/1.4.1/tutorials/text.html#verb-chain-tagging).

### Ülesanne 2. Verbiahelate statistika (2,5 p)

Võiks arvata, et ajalehtede arvamusartiklid eristuvad teistest artiklitest selle poolest, et keskenduvad rohkem väidete esitamisele ja ümberlükkamisele ning see väljendub ka vastavate artiklite verbiahelate kasutuses.
Kuivõrd see võiks kehtida, selle uurimiseks looge programm, mis teeb verbiahelate statistikat Eesti Päevalehe ja Postimehe arvamusartiklite ja tavaliste uudisartiklite põhjal ning väljastab tulemused võrdleva tabelina.

**2.1** (2 punkti) Programm loeb kataloogist `'1_aja_xml'` Eesti Päevalehe ja Postimehe faile (igas XML failis on üks ajalehenumber, st kõik sellel päeval vastavas lehes ilmunud artiklid) ning kogub kokku arvamusartiklid ja uudisartiklid.
Arvamusartiklid on mõlemas väljaandes rubriiginime _Arvamus_ all, uudisartiklite rubriiginimi on vastavalt kas _Eesti_ (Eesti Päevalehes) või _Uudised_ (Postimehes). 
Programm lisab kokkukogutud rubriikide artiklitele verbiahelate märgenduse, loendab verbiahelate tunnuseid ning väljastab tabeli, kus on mõlema rubriigi kohta toodud välja:

  1. eitusega verbiahelate %;
  2. modaalverbidega _'võima'_, _'tohtima'_ või _'pidama'_ verbiahelate % (ainult sellised ahelad, kus vastav verb esineb modaalverbi rollis);
  3. olevikuajaga verbiahelate %;
  4. mingi muu verbiahelate spetsiifilise tunnuse või struktuurielemendi loendamisel saadud % (valige see tunnus ise);
  
- | eituseid | modaalidega ahelaid | ...
 --- | --- | --- | ---
**Arvamused** |  x% | y% | ...
**Uudised** | z% | ... | ...

Detaile / soovitusi: 
  - tekstide sisselaadimiseks on tõenäoliselt kõige mugavam kasutada funktsiooni `parse_tei_corpora` (või `parse_tei_corpus`); Kindlasti tuleks määrata ka kodeering ('utf-8');
  - tabeli tegemisel ilmselt kõige mugavam kasutada `pandas.DataFrame`-i; üks viis, kuidas seda teha: tekitage algul tühi tabel, mille mõõtmed vastavad rubriikide ja uuritavate tunnuste arvule, ning tekstide analüüsi kõigus täitke see andmetega; kui te pole veel katsetanud andmete muutmist `DataFrame`-is, siis juhatust saab [siit](http://pandas.pydata.org/pandas-docs/stable/10min.html#setting);
  - modaalverbidega _'võima'_, _'tohtima'_ või _'pidama'_ ahelate loendamine:
     * eesmärgiks on leida ahelad, kus modaalverb on teise verbi tähenduse mõjutaja. Näiteks, lauses _'Me **peame säilitama** tasakaalu'_ on verb _**pidama**_ verbi _**säilitama**_ tähenduse mõjutaja, samas kui lauses _'Me **ei kavatse** enam restorani **pidada**'_ on küll verbiahel, mis sisaldab verbi _**pidama**_, aga verb _**pidama**_ pole seal mitte tähenduse mõjutaja, vaid sisuverb, ja seega modaalverbi tähenduses ei esine;
     * Kuna on mingi verb tähenduse mõjutaja ja kuna sisuverbi rollis, seda saab teada, kui uurida verbi positsiooni verbiahelas; üldine reegel on, et tähenduse mõjutaja on ahela eesotsas; erandiks on aga eitus ja liitaeg, mille puhul on modaalverb ahela teine liige;
     * Verbiahelas peaks modaalverbil olema vähemalt üks alluv; ilma alluvata modaalverbid jätta loendusest välja;


**2.2** (0,5 punkti) Katsetage 4. tunnuse valimisel erinevaid variante ja leidke tunnus, mille % erineb märgatavalt arvamusartiklites ja tavalistes uudisartiklites. Püüdke leida põhjendus, miks ühes rubriigis ilmneb tunnus rohkem kui teises, vajadusel uurige ka artiklite sisusid ja leidke illustreerivaid näiteid. Esitage põhjendus kirjalikult siin samas Notebookis.
    

### Ülesanne 3. Fokusseeritud vestluste tuvastamine (3 p)

Viimases ülesandes tegeleme taaskord foorumipostituste analüüsiga. 
Kataloogis `'2_foorumid_xml'` on veel Planeti alamfoorumite salvestusi: igas XML failis on ühe alamfoorumi teemad, mis on jagatud postitusteks ning postitused omakorda lauseteks.

Nimetagem _fokusseeritud vestluseks_ (edaspidi ka lihtsalt _vestluseks_) järjestikkuste postituste jada, kus vähemalt kaks postitust on erinevatelt kasutajatelt (ehk siis: tegemist on dialoogi, mitte monoloogiga) ning postitused on üsna keskendunud mingi alamteema arutamisele. 
Täpsemalt: _fokusseeritud vestlus_ on järjestikkuste osalausete jada, kus kõigi osalausete _sisuverb on sama_ (see peaks tagama  fokusseerituse) ning vähemalt kaks osalauset on erinevatest postitustest / erinevatelt kasutajatelt. 

Näited ( iga verbiahela lause on eraldi real, verbiahelatesse kuuluvad sõnad märgitud paksult ):

  * <u>teema:   Demokraatlik monarhia ? - 01. leht</u>  
    Folken : _Inglismaal demokraatlik monarhia , aga ma ei tea kas kuningas vms **valitakse** rahva poolt ._  
    Folken : _Maybe parlament **valib** ._  
    monstrum : _Kuningat **ei valita** , tainapead ._  
    
    ( sisuverb: _valima_, vestluse pikkus: 2 postitust, 3 lauset, 3 verbiahelat )


  * <u>teema:   Soovitusi lõpukirjandiks - 24. leht</u>  
    Hellion : _Wtf. alati **on saanud** oma toitu kaasa **võtta** ju ._  
    Hellion : _Ma ühe pudeli Nestea'd ja veits šokolaadi **võtan** ._  
    Hellion : _Megaeinet päris **võtma ei hakka** ._  
    troskal : _Mädarõigast ja leiba **võtke** kaasa ._  
    Cab Driver : _Ma **võtan** kala ._  
    
    ( sisuverb: _võtma_, vestluse pikkus: 3 postitust, 5 lauset, 5 verbiahelat )
    

Vestluse algus ei pea postituse algusega kokku langema, see võib alata ka postituse keskelt või lõpust.

Vestluse keskel võib olla ka selliseid osalauseid, millest ei tuvastatud ühtegi verbiahelat, aga vestluse alguses ja lõpus peavad kindlasti olema verbiahelaga osalaused, millel on sama sisuverb. Näide:

  * <u>teema:   mis motorollerid/tsiklid teil on ? - 01. leht</u>  
    Crazor : selline baaripukk **teeb** enamus tallinnas liikvel olevatele autodele nii kiirenduses kui tippkiiruses ara  
    giga : no ta **võib** ju ära **teha** , aga ikke sõidax enne auto või tsikliga , kui taburetiga ....  
    giga : nu muidugi , kui vanust piisavalt ..  
    Inspektor : Oot mismis ?  
    Inspektor : tippkiiruses **teen** enamusele ära ?  
    
    ( sisuverb: _tegema_, vestluse pikkus: 3 postitust, 5 lauset, 3 verbiahelat )
    
Ülesandeks on luua programm, mis analüüsib läbi kõik kaustas `'2_foorumid_xml'` olevad failid ning leiab sealt ülaltoodud kirjeldusele vastavad vestlused. 
Programmis on muudetav parameeter, mis kirjeldab vestluse miinimumpikkust verbiahelates: programm väljastab kõik vestlused, milles on vähemalt nii palju verbiahelaid kui miinimumpikkus.
Vaikimisi võiks miinimumpikkus olla 3 (nagu kõigis eeltoodud näidetes), lühemad ahelad meile huvi ei paku.

NB! Kuigi foorumis ulatuvad osad teemad üle mitme lehekülje, siis piisab, kui vestluseid otsitakse ainult ühe lehekülje piirest.

Vestluse väljastamisel kuvatakse kõigepealt teema nimi, seejärel vestlusesse kuuluvad verbiahelad, iga verbiahela lause eraldi reale.
On ok, kui liitlause puhul trükitakse üks lause mitu korda (kuna seal on mitu osalauset/verbiahelat). 
Iga lause algusesse lisatakse postitaja nimi. 
Tekstis verbiahelaid kuidagi esile tooma (nagu näidetes) ei pea, küll aga tuleks väljastada vestluse pikkus verbiahelates.
Lisaks peaks programm loendama väljastatud vestluste arvu -- nii igas failis, kui ka kõigi failide peale kokku -- ning väljastama (vastavalt siis faili või töö lõpus).

Vihjeid / soovitusi:

  * Andmete sisselugemine XML failist. Funktsioonid `parse_tei_corpus` ja `parse_tei_corpora` kahjuks selle ülesande puhul palju ei aita -- nende abil saab küll kätte laused (sisu `<s>`-märgendite vahel), aga kättesaamatuks jäävad postitajate nimed ja teemade pealkirjad. Seega tuleks andmete sisselugemisel kasutada hoopis teegi _BeautifulSoup_ vahendeid -- seda nii lausete kui ka muu info süstemaatiliseks lugemiseks;
  
  
  * Soovitav on postitusi analüüsida lause-kaupa ning esmalt koguda kokku analüüsi tulemused. Koguda järjendisse kõik ühe teema verbiahelad, sellises järjekorras, nagu need mööda postitusi liikudes ette tulevad. Ilmselt ei piisa ainult verbiahelate järjendisse salvestamisest, tuleks salvestada ka infot nende esinemiskonteksti kohta (postitaja, lause, sisuverb). Ning salvestada tuleks ka laused/osalaused, kus ei leidunud ühtegi verbiahelat, aga mis võivad siiski olla mõne vestluse keskel.
  
  
  * Kui ühe teema kõik ahelad on leitud, saabki asuda fokusseeritud vestluseid otsima. Selleks tuleb kõik teemaga seotud verbiahelad läbi vaadata ning iga verbiahela puhul uurida talle järgnevaid ahelaid: liikuda mööda kokkulangevaid sisuverbe ja liikumine katkestada, kui sisuverbid enam kokku ei lange. Kui niiviisi saadud ahela pikkus ületab parameetriga määratud miinimumpikkust, siis saab vestluse väljastada ekraanile.
  
  NB! Kuidagi tuleb ka meeles pidada, millised verbiahelad on juba eelmiste vestluste koosseisus väljastatud, et ei tekiks duplikaate. Selleks võib nt meeles pidada väljastatud verbiahelate indeksid järjendis ning hoiduda sama indeksiga ahela uuestiväljastamisest.


---

## Lisamaterjal: Serialiseerimine (_Pickle_)

Tekstilised formaadid nagu XML ja JSON on inimesele suhteliselt hästi loetavad, kuid nende lugemine/kirjutamine tähendab arvuti jaoks enamasti lisatööd: andmeid tuleb teisendada mälus oleva kuju ja tekstilise kuju vahel. 
Kui lood ise uue Pythoni klassi, siis ei saa selle objekte kohe JSON / XML kujul salvestada, vaid pead looma ka konverteri, mis teisendab objekte vastavatesse andmekujudesse (ja ka tagasi).

Pythoni andmestruktuuride ja objektide otse-salvestamiseks leidub ka alternatiivne ja kiirem viis, mida nimetatakse _serialiseerimiseks_. 
Sisuliselt tähendab serialiseerimine, et salvestamisel teisendatakse mälus olevad andmed binaarsele kujule ning failist laadimisel muudetakse need tagasi Pythoni mälus olevateks andmeteks.
Kuna andmete formaati sealjuures ei muudeta, on protsess enamasti suhteliselt kiire.
Samuti ei pea ise tegelema konverteri kirjutamisega: enamikku andmestruktuure, funktsioone, klasse kui ka klasside eksemplare / objekte on võimalik serialiseerida ilma, et peaks neid kuidagi teisendama.

Pythonis vastutab serialiseerimise eest teek [_pickle_](https://docs.python.org/3/library/pickle.html). Nagu teiste andmeformaatidegi puhul, on ka serialiseerimise tulemused platvormist sõltumatud ehk serialiseerimise abil tekitatud fail peaks olema suvalisel (Pythoniga) masinal loetav. Binaarse kuju miinuseks on aga kahjuks see, et failis olevad andmed pole enam inimsilmale loetavad.

Näitena serialiseerime _rows_ muutuja.

In [13]:
rows = [{'a':5, 'b':3}, {'c':4, 'd':1 }]

import pickle
with open('serialized_ppl.pkl','wb') as fout:
    pickle.dump(rows, fout)

Paneme tähele, et fail tuleb avada nii lugemiseks kui kirjutamiseks _binary mode_'is (`'wb'` ja `'rb'`).

Võite iseseisvalt uurida, milline on tekitatud faili `'serialized_ppl.pkl'` sisu. (Eriti loetav see pole.)

Funktsiooni `pickle.load` abil saame serialiseeritud sisu jälle Pythonisse tagasi lugeda:

In [14]:
with open('serialized_ppl.pkl','rb') as fin:
    new_rows = pickle.load(fin)
    print(new_rows)

[{'a': 5, 'b': 3}, {'d': 1, 'c': 4}]


Vaikimisi saab _pickle_ hakkama paljude standardteekide klasside serialiseerimisega ning ka endakirjutatud teekidega, mis kasutavad  standardteegi objekte.

### EstNLTK `Text` objektide serialiseerimisest

Ka EstNLTK `Text` objektide salvestamiseks/laadimiseks on serialiseerimine üsna kiire viis.
Siin on aga üks tehniline nüanss: **kõik `Text` objektid pole otsemaid serialiseeritavad**.
Kui `Text`-ile on lisatud ajaväljendite, osalausete ja/või verbiahelate märgendus, siis kohe selle serialiseerimine ei õnnestu -- `pickle.dump` meetod annab veateate stiilis `TypeError: cannot serialize '_io.BufferedReader' object`.
Põhjus on selles, et ajaväljendite tuvastaja ja osalausestaja (millest omakorda sõltub ka verbiahelate tuvastaja) kasutavad teksti analüüsimisel Pythoni-väliseid Java komponente ning klassid, mis teostavad suhtlust nende komponentidega, pole serialiseeritavad.

Probleemile leidub ka lahendus.
Äsjalisatud märgendustega `Text` objekt tuleb ümber kirjutada nii, et säilivad märgendused, aga eemaldatakse mitte-serialiseeritavaid alamkomponente sisaldava märgendajad (ehk siis: ajaväljendite tuvastaja, osalausestaja, verbiahelate tuvastaja). 
Kui märgendused on juba lisatud, pole märgendajaid ka enam tarvis ning need võib vabalt eemaldada.

Vaatame lahendust ühe näite varal lähemalt.

In [15]:
# Loome serialiseeritava teksti
from estnltk import Text
text = Text('Küll homme armastab see, kes ei armastanud eile.')
# Lisame kõik märgendused (sh ajaväljendid, osalaused ja verbiahelad)
text = text.tag_all()

Kohe selle teksti serialiseerimine ei õnnestu. Võid ise järgi katsetada:

    import pickle
    with open('test_serialized_text.pkl','wb') as fout:
         pickle.dump(text, fout)
         
Küll aga saame luua vana teksti põhjal uue teksti, mis sisaldab samu märgendusi, aga ei sisalda märgendajaid:

In [17]:
# Loome uue teksti, mis sisaldab samu märgendusi, aga mitte samu märgendajaid
new_text = Text( text )

Kui loome uue `Text` objekti eelkirjeldatud viisil, siis kopeeritakse vanast objektist kogu sõnastiku sisu (ehk kõik märgendused), aga kopeerimata jäetakse vana teksti küljes olevad "märgendajad" (kuna märgendajad pole otseselt sõnastikus, vaid on klassi `Text` varjatud muutujate all).
Seega puuduvad uuel `Text` objektil komponendid, mille serialiseerimine ei õnnestu.

Serialiseerime uue teksti:

In [18]:
import pickle
with open('test_serialized_text.pkl','wb') as fout:
    pickle.dump(new_text, fout)

Laeme teksti sisse ja uurime, kas märgendused säilisid:

In [19]:
with open('test_serialized_text.pkl','rb') as fin:
    loaded_text = pickle.load(fin)

print(loaded_text)
print('  timexes:', loaded_text.timex_texts)
print('  clauses:', loaded_text.clause_texts)
print('  verb_chains:', loaded_text.verb_chain_texts)

Küll homme armastab see, kes ei armastanud eile.
  timexes: ['homme', 'eile']
  clauses: ['Küll homme armastab see,', 'kes ei armastanud eile.']
  verb_chains: ['armastab', 'ei armastanud']


---